# Steps to Add Recordings and Pictures

---

NOTE: instructions below assume you have downloaded all mp3 files in the `audio/mp3` folder.

---

*1. For images...*

Look for a new species not in `csv/chosen.csv` and search for a good picture in [ebird](https://ebird.org/) (append the species name to:  `https://ebird.org/species/`).

Download the picture from the [Macaulay Library](https://macaulaylibrary.org/) and make sure to note the copyright owner and URL.

Add a line to `csv/pic_metadata.csv` filling out all the info.

Save the picture in `img/ebird/` with its bird species (`ebird_code`) name.

Make sure the picture has dimensions 480px (width) by 320px (height).

*2. For bird calls...*

Find a good recording of the species in `audio/mp3` that is at least 10 seconds long and note the `xc_id`.

Check the recording at [xeno-canto](https://xeno-canto.org/) by appending its xeno-canto catalogue number to this main URL.

Add a new line to `csv/chosen.csv` with the new bird species `ebird_code` and specific recording `xc_id` (just `code`).

*3. Finally*

You can do the above steps any number of times. When done adding recordings and images, close all `csv` files and **Run All** cells below, then check the output of the cells to make sure numbers make sense.

In [3]:
import os
import subprocess
import pandas as pd
from PIL import Image

In [4]:
# chosen mp3s
chosen_path =  os.path.join("csv", "chosen.csv")
chosen_df = pd.read_csv(chosen_path)
npecies = len(chosen_df['ebird_code'].unique())

print(f'There are {npecies} species\
 and {len(chosen_df)} recordings chosen.')

There are 73 species and 87 recordings chosen.


In [5]:
# wav files and Xcodes in audio/
wav_dir = os.path.join("audio", "wav")

def get_waves():
    
    subdirs, waves, codes = [], [], []

    for subdir in os.listdir(wav_dir):
        subdirs.append(subdir)
        for i, wav in enumerate(os.listdir(os.path.join(wav_dir, subdir))):
            waves.append(os.path.join(wav_dir, subdir, wav))
            codes.append(wav.split('.')[0][2:])

    print(f'There are {len(subdirs)} bird species\
 and {len(waves)} recordings in audio/wav/')
    
    return subdirs, waves, codes

In [6]:
# PRE CHECK
subdirs, waves, codes = get_waves()

There are 71 bird species and 85 recordings in audio/wav/


In [7]:
# FIX AUDIO
# ---------
# get outstanding mp3 files and convert to wav files
# ASSUMPTIONS:
# 1. new ecode (new species) recording
# 2. only 1 recording per species (ecode)

mp3_dir = os.path.join("audio", "mp3")
wav_dir = os.path.join("audio", "wav")

# (assumption 2) only 1 xcode
for ecode, xcode in zip(chosen_df['ebird_code'], chosen_df['code']):
    
    # (assumption 1) new ecode
    if ecode not in subdirs:
        
        mp3_subdir = os.path.join(mp3_dir, ecode)
        wav_subdir = os.path.join(wav_dir, ecode)
        
        try:
            os.stat(wav_subdir)
        except FileNotFoundError:
            os.makedirs(wav_subdir)

        mp3 = ''.join([''.join(['XC', str(xcode)]), '.mp3'])
        wav = ''.join([''.join(['XC', str(xcode)]), '.wav'])

        mp3_filepath = os.path.join(mp3_subdir, mp3)
        wav_filepath = os.path.join(wav_subdir, wav)

        subprocess.call([
            'ffmpeg', '-hide_banner',
            '-loglevel', 'quiet',
            '-i', mp3_filepath, wav_filepath
        ])

In [8]:
# POST CHECK
subdirs, waves, codes = get_waves()

There are 73 bird species and 87 recordings in audio/wav/


In [9]:
# FIX PICS
# --------
pic_path = os.path.join("img", "ebird")

def get_jpgs():
    jpgs = pngs = 0
    for img in os.listdir(pic_path):
        ebird_code, ext = img.split(".")
        if ext == "jpg":
            jpgs += 1
        elif ext == "png":
            pngs += 1
        else:
            pass
        
    print(f'There are {len(os.listdir(pic_path))} pictures,\
 {jpgs} jpg(s) and {pngs} png(s).')

In [10]:
get_jpgs()

There are 73 pictures, 2 jpg(s) and 71 png(s).


In [11]:
# convert any jpg(s) to png(s) and delete them
for img in os.listdir(pic_path):
    ebird_code, ext = img.split(".")
    if ext == "jpg":
        jpg = os.path.join(pic_path, img)  
        image = Image.open(jpg)  
        png = os.path.join(pic_path, ".".join([ebird_code, "png"]))
        image.save(png)
        os.remove(jpg)

In [12]:
get_jpgs()

There are 73 pictures, 0 jpg(s) and 73 png(s).


---